# Install the package

First, you need to install the package from github and import it:

In [1]:
# Install the bulkhours package
!rm -rf bulkhours && git clone https://github.com/guydegnol/bulkhours.git

# import the bulkhours package
import bulkhours

# Generate header links
bulkhours.generate_header_links("examples/1_Simple_Exercice_Automation.ipynb")

Cloning into 'bulkhours'...
remote: Enumerating objects: 5159, done.
remote: Counting objects: 100% (758/758), done.
remote: Compressing objects: 100% (368/368), done.
remote: Total 5159 (delta 474), reused 642 (delta 361), pack-reused 4401
Receiving objects: 100% (5159/5159), 40.86 MiB | 10.84 MiB/s, done.
Resolving deltas: 100% (3311/3311), done.



[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/guydegnol/bulkhours/blob/main/examples/1_Simple_Exercice_Automation.ipynb) [![Open In Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/guydegnol/bulkhours/blob/main/examples/1_Simple_Exercice_Automation.ipynb) [![GitHub](https://badgen.net/badge/icon/Open%20in%20Github?icon=github&label)](https://github.com/guydegnol/bulkhours/blob/main/examples/1_Simple_Exercice_Automation.ipynb) [![Visual Studio](https://badgen.net/badge/icon/Open%20in%20Visual%20Studio?icon=visualstudio&label)](https://vscode.dev/github/guydegnol/bulkhours/blob/main/examples/1_Simple_Exercice_Automation.ipynb) [![CC-0 license](https://img.shields.io/badge/License-CC--0-blue.svg)](https://creativecommons.org/licenses/by-nd/4.0)
                            

# Create my first exercice


**Exercise**: Using `numpy.exp`, implement the $\sigma(z) = \frac{1}{1 + e^{-z}}$ to make predictions. The expected solution is in the following cell:


In [2]:
import numpy as np

def sigmoid(z):
  s = 1 / (1 + np.exp(-z))
  return s

# Set a unique id for the exercice

To identify an exercice, we need to define an id for each cell. Here the id of the exercice is ```my_first_exercice```:

In [3]:
%%evaluation_cell_id -i my_first_exercice

def sigmoid(z):
  s = 1 / (1 + np.exp(-z))
  return s

for z in [1, 3, 7]:
  print(f"sigmoid({z})={sigmoid(z)}")

# To be able to use your the evaluation tools, you need to identify yourself with the following command:
bulkhours.init_env(email="name@institute.ac")  # To be run in an initisalization cell

# You can also use the following command to get more information:
help(bulkhours.init_env)
    


The execution of the previous cell should have failed. **You need to identify yourself first** :



In [8]:
bulkhours.init_env(email="dave.gahan@bulkhours.org")

Import BULK Helper cOURSe (version='5.2.15'🚀', ⚠️ in admin/teacher🎓 mode⚠️):⚠️Database is local (security_level=0). Export your config file if you need persistency.⚠️
(db, subject, virtual_room, nb_id, user) = (free1.json, '', room1, '', dave.gahan@bulkhours.org🎓)


💡**In practice, you should use secured a access with identification checking to avoid security issues**. For example, this is a typical identification used in course:
```
# A secured access to the database for the teacher
bulkhours.init_env(email="dave.gahan@bulkhours.org", database="bkloud@prems/teacher::eNqlVtmSo8gV_ZV-cd...")

# A secured access to a virtual class database for students
bulkhours.init_env(email="dave.gahan@bulkhours.org", database= "bkloud@prems/winterfell_student::eNqVVtmyo8gV_JV-cfRMaLpB7...")
```



# Get the answer from another notebook

Normaly, all the following code should be runned in a teacher notebook.

In demo mode, you can simply switch to the teacher mode calling the
```bulkhours.init_env``` with the ```is_admin=True``` argument/


In [10]:
bulkhours.init_env(email="johnny.cash@bulkhours.org", is_admin=True)

Import BULK Helper cOURSe (version='5.2.15'🚀', ⚠️ in admin/teacher🎓 mode⚠️):⚠️Database is local (security_level=0). Export your config file if you need persistency.⚠️
(db, subject, virtual_room, nb_id, user) = (free1.json, '', room1, '', johnny.cash@bulkhours.org🎓)



In real life, this code should be runned from the teacher notebook only

In [11]:
bulkhours.admin.evaluate("my_first_exercice", user="dave.gahan@bulkhours.org", show_correction=False)

'L'étudiant dave.gahan@bulkhours.org est inconnu. Ajouter le depuis le menu dashboard:
bulkhours.admin.dashboard()
Cache file (/content/bulkhours/data/cache/room1/admin__my_first_exercice.json) of my_first_exercice à 2023-06-30 09:46:05 pour /room1. Mise à jour locale



HTML(value="<b><font face='FiraCode Nerd Font' size=4 color='red'>Code (Dave.g)📚<font></b>", layout=Layout(hei…

def sigmoid(z):
  s = 1 / (1 + np.exp(-z))
  return s

for z in [1, 3, 7]:
  print(f"sigmoid({z})={sigmoid(z)}")

HTML(value="<b><font face='FiraCode Nerd Font' size=4 color='red'>Execution (Dave.g)💻<font></b>", layout=Layou…

sigmoid(1)=0.7310585786300049
sigmoid(3)=0.9525741268224334
sigmoid(7)=0.9990889488055994


Output()

# Automation of the grade


Now we can build a correction for this exercice

In [12]:
%%evaluation_cell_id -i my_first_exercice
import numpy as np

def sigmoid(z):
  s = 1 / (1 + np.exp(-z))
  return s

for z in [1, 3, 7]:
  print(f"sigmoid({z})={sigmoid(z)}")

def student_evaluation_function():
    score = 0
    score += bulkhours.is_equal(sigmoid(np.array([1, 3, 7])), max_score=5)
    score += bulkhours.is_equal(sigmoid(3), teacher.sigmoid(3), max_score=5)
    return score


Output()